<a href="https://colab.research.google.com/github/profteachkids/STEMUnleashed2023/blob/main/tSNE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install datasets
!pip install einops

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 612.0 kB/s eta 0:00:00


In [55]:
import jax.numpy as jnp
import jax
jax.config.update("jax_enable_x64", True)
import numpy as np
import pandas as pd
from plotly.subplots import make_subplots
import plotly.express as px
from einops import rearrange
from scipy.optimize import minimize

In [3]:
from datasets import load_dataset

dataset = load_dataset("mnist", split="train")

Extracting data files:   0%|          | 0/4 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/60000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/10000 [00:00<?, ? examples/s]

Dataset mnist downloaded and prepared to /root/.cache/huggingface/datasets/mnist/mnist/1.0.0/9d494b7f466d6931c64fb39d58bb1249a4d85c9eb9865d9bc20960b999e2a332. Subsequent calls will reuse this data.


In [63]:
N=300
imgs=np.stack(dataset[:N]['image'])
labels=dataset[:N]['label']

In [64]:
plot_img=rearrange(imgs[:9], '(b1 b2) h w -> (b1 h) (b2 w)', b1=3)

In [65]:
fig=px.imshow(plot_img, color_continuous_scale='gray')
fig.update_layout(width=300,height=300,template='plotly_dark',coloraxis_showscale=False)
fig.show()

In [66]:
x=rearrange(imgs, 'b h w -> b (h w)').astype(np.float64)

In [82]:

twosigmasq = np.mean(np.sum((x[:,None,:]-x[None,:,:])**2,axis=-1))/10
expnegdelta=np.exp(-np.sum((x[:,None,:]-x[None,:,:])**2,axis=-1)/twosigmasq)+1e-15
np.fill_diagonal(expnegdelta,0.)
pij=expnegdelta/np.sum(expnegdelta,axis=1)
pij=jnp.asarray((pij+pij.T)/2/N)



In [83]:
def KL_scipy(v):
    y=v.reshape(-1,2)
    qij=1/(1+jnp.sum((y[:,None,:]-y[None,:,:])**2,axis=-1))
    qij=qij.at[jnp.diag_indices(N)].set(0.)
    qij = qij/jnp.sum(qij)
    return jnp.sum(pij[nodiagmask]*jnp.log(pij[nodiagmask]/qij[nodiagmask]))

KL_scipy_jit = jax.jit(KL_scipy)
KL_scipy_grad=jax.jit(jax.grad(KL_scipy))

In [84]:
rng=np.random.RandomState(123)
y0=jnp.asarray(rng.uniform(-10,10,size=(N,2))).flatten()
nodiagmask=np.full((N,N),True)
np.fill_diagonal(nodiagmask,False)
sol=minimize(KL_scipy_jit,y0,method='L-BFGS-B',jac=KL_scipy_grad, tol=1e-10)
sol

  message: CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH
  success: True
   status: 0
      fun: 0.4653148065653369
        x: [-1.378e+01  1.901e+01 ... -4.028e+01 -1.215e+01]
      nit: 1837
      jac: [-1.724e-08  2.002e-09 ...  7.665e-09  3.536e-09]
     nfev: 1899
     njev: 1899
 hess_inv: <600x600 LbfgsInvHessProduct with dtype=float64>

In [88]:
def KL(v):
    y=v.reshape(-1,2)
    qij=1/(1+jnp.sum((y[:,None,:]-y[None,:,:])**2,axis=-1))
    qij=qij.at[jnp.diag_indices(N)].set(0.)
    qij = qij/jnp.sum(qij)
    return jnp.sum(pij[nodiagmask]*jnp.log(pij[nodiagmask]/qij[nodiagmask]))

KLjit = jax.jit(KL)
KLgrad=jax.jit(jax.grad(KL))

In [90]:
gsum=0
yorig=y0
ydefmax=1e-3

In [92]:
#Distance over Gradient - Stochastic Gradient Descent
#https://arxiv.org/pdf/2302.12022.pdf

for i in range(int(5e3)):
    g=KLgrad(y0)
    gsum+=jnp.sum(g**2)
    eta= ydefmax/jnp.sqrt(gsum)
    y1 = y0 - eta*g
    ydelta=jnp.linalg.norm(y1-yorig)
    ydefmax = ydelta if ydelta > ydefmax else ydefmax
    y0=y1
    if i % 1000 == 0:
        print(i, KL(y0), eta)

0 0.5701858175097889 458.0583625894854
1000 0.5676926848915418 442.529028349111
2000 0.5667474603958271 429.1018796626659
3000 0.5651047976002448 417.68658630970737
4000 0.5644327665247598 407.9872551589513


In [94]:
df2=pd.DataFrame(y0.reshape(-1,2))
# df2=pd.DataFrame(sol.x.reshape(-1,2))
df2['label']=labels
df2['label']=df2['label'].astype('category')

In [95]:
fig2=px.scatter(df2,x=0,y=1,color='label', color_discrete_sequence=px.colors.qualitative.G10)
fig2.update_layout(width=500,height=500,template='plotly_dark')